In [ ]:
import pandas as pd
import folium
from time import sleep
from os import path, listdir
import numpy as np

In [ ]:
x, y = 69.3, 16
vx, vy = 0.00001, 0.00005
trajectory = [(x, y)]

lon_low = 5
lon_high = 30
lat_low = 60
lat_high = 80

In [ ]:
x, y = 69.31, 16
m = folium.Map(location=[x, y], zoom_start=13)
marker = folium.Marker([x, y])
marker.add_to(m)
mapfile = 'gps_map.html'

file_times = [(f, path.getmtime(f)) for f in listdir('.') if path.isfile(f) and f[-3:] == 'csv']

sorted_files = sorted(file_times, key=lambda x: x[1], reverse=True)
data_source = sorted_files[0][0]

while True:
    df = pd.read_csv(data_source, comment='#')
    lons = (pd.to_numeric(df['lon_deg']) + pd.to_numeric(df['lon_min']) / 60 + pd.to_numeric(df['lon_sec']) / 3600).to_numpy()
    lats = (pd.to_numeric(df['lat_deg']) + pd.to_numeric(df['lat_min']) / 60 + pd.to_numeric(df['lat_sec']) / 3600).to_numpy()
    mask = (lons<lon_high)&(lons>lon_low)&(lats<lat_high)&(lats>lat_low)
    lons = lons[mask]
    lats = lats[mask]
    trajectory = np.array([lats, lons]).T
    marker.location = list(trajectory[-1,:])
    trajectory_line = folium.PolyLine(locations=list(trajectory), color='blue')
    m.add_child(trajectory_line)
    m.save(mapfile)
    sleep(5)